# Load Packages for Running this Script

In [ ]:
%matplotlib notebook
import pandas as pd
import numpy as np #NumPy version >=1.18.5 is required. Install using `pip install numpy=1.24.1` or `conda install numpy=1.24.1` (withou quotes!)
from scipy.signal import savgol_filter
import matplotlib.pyplot as plt
from matplotlib.widgets import RectangleSelector

# Load Your "CSV" File

In [ ]:
# Load CSV data, using the second and third rows as the header
df = pd.read_csv(r"C:\Users\Farha\Downloads\Mouse9_MetalExposed_T-maze_12-14-23DLC_resnet101_TMZ_Model_V2Dec16shuffle1_157000.csv", header=[1, 2])
print(df.columns)

# Create and Define Heads for body parts and X, Y , and Likelihood. 

In [ ]:
# Create a dictionary to map body part names to coordinate columns
body_part_data = {}
# Get unique body parts from the first level of the MultiIndex (excluding 'bodyparts')
unique_body_parts = df.columns.get_level_values(0).unique().tolist()
unique_body_parts.remove('bodyparts')

for part in unique_body_parts:
    # Use the MultiIndex to access the data
    body_part_data[part] = {
        "x": df.loc[:, (part, 'x')].to_numpy(),
        "y": df.loc[:, (part, 'y')].to_numpy(),
        "likelihood": df.loc[:, (part, 'likelihood')].to_numpy()
    }

for part in body_part_data.keys():
    print(f"body parts: {part}")

# Define Region of Interest (ROI) in this Block

In [ ]:
# Initialize variables for ROI
roi_coords = []

# Function to be called when the rectangle is drawn
def onselect(eclick, erelease):
    x1, y1 = eclick.xdata, eclick.ydata
    x2, y2 = erelease.xdata, erelease.ydata
    roi_coords.append((x1, y1, x2, y2))
    print(f"ROI: ({x1}, {y1}), ({x2}, {y2})")
    
# Plot the data for the 'Thorax' body part
fig, ax = plt.subplots()
Head_x = body_part_data['Nose']['x']
Head_y = body_part_data['Nose']['y']
ax.plot(Head_x, Head_y, label='Nose')

# Set useblit=True on most backends for enhanced performance
toggle_selector = RectangleSelector(ax, onselect, useblit=True,
                                    button=[1],  # Left mouse button
                                    minspanx=5, minspany=5,
                                    spancoords='pixels',
                                    interactive=True)

plt.show()

# Check if Body Parts are Present in the Region of Interest (ROI) and Calculate Distances, Velocities, and Time Spent in the ROI.

In [ ]:
# Initialize the dictionaries
distances = {}
roi_time_spent = {}

# Define function to check if points are in the ROI
def is_in_roi(x, y, roi):
    x1, y1, x2, y2 = roi
    return (x1 < x < x2) and (y1 < y < y2)

# Function to calculate distance between two points
def calculate_distance(part_data_1, part_data_2, index):
    return np.sqrt((part_data_1["x"][index] - part_data_2["x"][index])**2 +
                   (part_data_1["y"][index] - part_data_2["y"][index])**2)

# Function to update time spent in ROI
def update_time_spent(roi_time_spent, part_pair, condition):
    if condition:
        roi_time_spent[part_pair] += 1

# Function to calculate total time spent in ROI
def calculate_total_time_in_roi(roi_time_spent):
    return sum(roi_time_spent.values())

# User-configurable list of body part pairs to track
body_part_pairs = [("Thorax", "Head")]  # Example pair; user can change body parts to match their own experiments

# Calculate distances and time spent in ROI for each body part pair
for part1, part2 in body_part_pairs:
    distances[f"{part1}-{part2}"] = [calculate_distance(body_part_data[part1], body_part_data[part2], index)
                                      for index in range(len(df))]
    roi_time_spent[f"{part1}-{part2}"] = 0

    # Update time spent in ROI for each frame
    for index in range(len(df)):
        update_time_spent(roi_time_spent, f"{part1}-{part2}",
                          roi_coords and is_in_roi(body_part_data[part1]["x"][index], body_part_data[part1]["y"][index], roi_coords[-1]))

# Calculate and print the total time spent in ROI
total_time_in_roi = calculate_total_time_in_roi(roi_time_spent)
print(f"Total time spent in ROI: {total_time_in_roi} frames")


# Filter the Data with Savitzky-Golay Filter



In [ ]:
# Assuming body_part_data is a dictionary with keys as body parts
body_parts = list(body_part_data.keys())

# Parameter for frame rate
frame_rate = 30  # Replace with your actual frame rate

# Initialize the dictionary for velocities
velocities = {}

# Calculate velocities and apply Savitzky-Golay filter for smoothing
for part in body_parts:
    # Ensure there are enough data points for velocity calculation
    if len(body_part_data[part]["x"]) > 1:
        # Calculate velocity as the difference in position over time
        velocities[part] = np.diff(body_part_data[part]["x"]) / frame_rate
        
        # Determine the window length for the Savitzky-Golay filter
        # It must be odd and less than the size of the data
        window_length = min(15, len(velocities[part]) // 2 * 2 + 1)
        
        # Apply the Savitzky-Golay filter if the window length is valid
        if window_length > 2:  # At least 3 points are needed to apply the filter
            velocities[part] = savgol_filter(velocities[part], window_length=window_length, polyorder=3)
        else:
            print(f"Not enough data points to apply Savitzky-Golay filter for {part}.")
    else:
        print(f"Not enough data points to calculate velocity for {part}.")


# Calculate Average Velocity of Each Body Parts

In [ ]:
# Assuming velocities is a dictionary with keys as body parts and values as velocity data
# Initialize the dictionary for average velocities
average_velocities = {}

# Iterate over each body part to calculate and print the average velocity
for part in body_parts:
    # Calculate the average velocity for each body part
    average_velocity = np.mean(np.abs(velocities[part]))
    # Store the average velocity in the dictionary
    average_velocities[part] = average_velocity
    # Print the average velocity for each body part
    print(f"Average velocity for {part}: {average_velocity:.2f} units/frame")


# Entries and Exits when Two Body Parts Simultaniously are present in the Regions of Interest for a specified number of frames. 

In [ ]:
# Define the body parts you want to track
parts_to_track = ['Nose', 'Head']  # User can replace with the body parts they're interested in

# debound_frame assings the limit for the minimum consecutive frames that the mouse must have entered the ROI before it is counted as entry.
def track_multi_part_entrances_exits(body_part_data, parts, roi, debounce_frames=30):
    inside_roi = False
    entrances = 0
    exits = 0
    inside_counter = 0  # Counter for consecutive frames inside the ROI
    outside_counter = 0  # Counter for consecutive frames outside the ROI
    
    # Get the coordinates for the body parts
    coords = {part: body_part_data[part] for part in parts}
    
    for i in range(len(body_part_data[parts[0]]['x'])):
        # Check if all specified body parts are in the ROI
        currently_inside = all(is_in_roi(coords[part]['x'][i], coords[part]['y'][i], roi) for part in parts)
        
        if currently_inside:
            inside_counter += 1
            outside_counter = 0  # Reset outside counter
        else:
            outside_counter += 1
            inside_counter = 0  # Reset inside counter
        
        # Count entrance only if body parts have been inside for enough consecutive frames
        if inside_counter >= debounce_frames and not inside_roi:
            entrances += 1
            inside_roi = True
            inside_counter = 0  # Reset inside counter after counting entrance
        
        # Count exit only if body parts have been outside for enough consecutive frames
        if outside_counter >= debounce_frames and inside_roi:
            exits += 1
            inside_roi = False
            outside_counter = 0  # Reset outside counter after counting exit
    
    # Print the number of entrances and exits
    print(f"Entrances: {entrances}, Exits: {exits}")
    return entrances, exits

# Example usage
if roi_coords:  # Ensure that an ROI has been selected
    last_roi = roi_coords[-1]  # Use the last ROI drawn
    entrances, exits = track_multi_part_entrances_exits(body_part_data, parts_to_track, last_roi)
    print(f"The mouse entered the ROI {entrances} times and exited {exits} times with both 'Nose' and 'Head' inside.")


# Define a Threshold for Max Distance a Body Part Can move prior to calculating Distance Moved by body parts

In [ ]:
# Define the maximum distance a body part can move per frame (this should be based on your specific data)
max_distance_per_frame = 100  # Example threshold value for the distance the mouse's bodyparts can move, adjust based on your data. Units are pixels.
frame_rate = 30  # Frames per second, adjust based on your data

def detect_jumps(x_coords, y_coords, max_distance):
    jumps = []
    for i in range(1, len(x_coords)):
        distance = np.sqrt((x_coords[i] - x_coords[i-1])**2 + (y_coords[i] - y_coords[i-1])**2)
        if distance > max_distance:
            jumps.append(i)
    return jumps

def calculate_filtered_distance(body_part_data, part_name, max_distance_per_frame):
    total_distance = 0
    x_coords = body_part_data[part_name]['x']
    y_coords = body_part_data[part_name]['y']
    jumps = detect_jumps(x_coords, y_coords, max_distance_per_frame)
    
    for i in range(1, len(x_coords)):
        if i not in jumps:
            distance = np.sqrt((x_coords[i] - x_coords[i-1])**2 + (y_coords[i] - y_coords[i-1])**2)
            total_distance += distance
    
    return total_distance, jumps

# Calculate the filtered total distance moved for each body part
body_parts = ['Head', 'Nose']  # Add 'TailBase' or other parts as needed
distances = {}
jumps_detected = {}

for part in body_parts:
    distances[part], jumps_detected[part] = calculate_filtered_distance(body_part_data, part, max_distance_per_frame)

# Sum the distances moved by all tracked body parts in units
total_filtered_distance_moved_in_units = sum(distances.values())

# Print the filtered total distance moved for each body part and the total in units
for part, distance in distances.items():
    print(f"Filtered total distance moved by the mouse's {part}: {distance:.0f} units")
    if jumps_detected[part]:
        print(f"Jumps detected for {part} at frames: {jumps_detected[part]}")

print(f"Filtered total distance moved together by the mouse's {' and '.join(body_parts)}: {total_filtered_distance_moved_in_units:.0f} units")
